In [2]:
from random import seed
from random import random
from math import exp
import numpy as np
import pandas as pd
import textwrap
import timeit
import binascii
import json
import zlib, base64

In [3]:
# -*- coding: utf-8 -*-
import json

# Make it work for Python 2+3 and with Unicode
import io
try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [4]:
def addZeros(strr, n): 
    for i in range(n): 
        strr = "0" + strr 
    return strr 

def getXOR(a, b): 
    aLen = len(a) 
    bLen = len(b) 


    if (aLen > bLen): 
        b = addZeros(b, aLen - bLen) 
    elif (bLen > aLen): 
        a = addZeros(a, bLen - aLen) 
 
    lenn = max(aLen, bLen); 

    res = "" 
    for i in range(lenn): 
        if (a[i] == b[i]): 
            res += "0"
        else: 
            res += "1"
    return res 

In [5]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [6]:
#initializing the network
def initialize_network(parameters):
    network = list()
    for j in range(1,len(parameters)):
        layer = [{'weights':[random() for i in range(parameters[j-1] + 1)]} for i in range(parameters[j])]
        network.append(layer)
    return network

In [7]:
# Make a prediction with a network
def predict(network, row, encr):
    outputs = forward_propagate(network, row, encr)
    k = [round(i) for i in outputs]
    if encr == 0:
        return k
    else:
        s = ""
        k = list(map(str,k))
        k = s.join(k)
        return k

In [8]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [9]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [10]:
# Forward propagate input to a network output
def forward_propagate(network, row, getEncryption):
    count = 0
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        if count == 1 and getEncryption == 1:
            print("io")
            return new_inputs
        inputs = new_inputs
        count += 1
  
    return inputs

In [11]:
# Update network weights with error
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            count += 1
            outputs = forward_propagate(network, row, 0)
            expected = row[-1]
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        if count % 100 == 0:
            print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        count += 1

In [12]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [13]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [14]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    #print(inputs)
    activation = weights[-1]
    for i in range(len(weights)-1):
        #print(i)
        #print(type(weights[i]))
        #print(type(inputs[i]))
        activation += weights[i] * inputs[i]
    return activation

In [15]:
#batch training
# Train a network for a fixed number of epochs
def train_network_batch(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    k = 0
    batch = 2
    for i in range(batch):
        print('> Batch = %d' % (i+1))
        for epoch in range(n_epoch):
            sum_error = 0
            for row in train[k:k+10]:
                count += 1
                outputs = forward_propagate(network, row, 0)
                expected = row[-1]
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)
            if count % 100 == 0:
                print('    epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
            count += 1
        k += 10

In [25]:
# Read JSON file
with open('../network/network.json') as data_file:
    network = json.load(data_file)

In [31]:
# Read JSON file
with open('../datasets/languageDecryptor.json') as data_file:
    dataset = json.load(data_file)

with open('../datasets/languageDecryptor.json') as data_file:
    temp = json.load(data_file)

In [30]:
dataset

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [18]:
for i in range(100):
    k = temp[i][-1]
    k.append(temp[i][:-1])
    dataset.append(k)

In [19]:
dataset

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [20]:
seed(1)
network = initialize_network([8,16,16,8])

In [21]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(dataset[0][-1])
start = timeit.default_timer()
train_network(network, dataset[0:25], 0.5, 10, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

time = 0.10773168100058683


In [22]:
c = 0
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        c += 1
print(str(c)+"% error")

200% error


In [24]:
len(dataset)

200

In [23]:
c = 0
new_dataset = list()
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        print("index = %d" % (c))
        print(k)
        print(j)
        print()
    c += 1

index = 0
11111111
00110000

index = 1
11111111
00110001

index = 2
11111111
00110010

index = 3
11111111
00110011

index = 4
11111111
00110100

index = 5
11111111
00110101

index = 6
11111111
00110110

index = 7
11111111
00110111

index = 8
11111111
00111000

index = 9
11111111
00111001

index = 10
11111111
01100001

index = 11
11111111
01100010

index = 12
11111111
01100011

index = 13
11111111
01100100

index = 14
11111111
01100101

index = 15
11111111
01100110

index = 16
11111111
01100111

index = 17
11111111
01101000

index = 18
11111111
01101001

index = 19
11111111
01101010

index = 20
11111111
01101011

index = 21
11111111
01101100

index = 22
11111111
01101101

index = 23
11111111
01101110

index = 24
11111111
01101111

index = 25
11111111
01110000

index = 26
11111111
01110001

index = 27
11111111
01110010

index = 28
11111111
01110011

index = 29
11111111
01110100

index = 30
11111111
01110101

index = 31
11111111
01110110

index = 32
11111111
01110111

index = 33
11111111


In [197]:
with open('../datasets/languageDecryptor.json') as data_file:
    temp1 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp2 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp3 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp4 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp5 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp6 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp7 = json.load(data_file)

In [17]:
temp1

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [198]:
for i in range(100):
    temp1[i][-1] = temp1[i][-1][1:2]
    temp2[i][-1] = temp2[i][-1][2:3]
    temp3[i][-1] = temp3[i][-1][3:4]
    temp4[i][-1] = temp4[i][-1][4:5]
    temp5[i][-1] = temp5[i][-1][5:6]
    temp6[i][-1] = temp6[i][-1][6:7]
    temp7[i][-1] = temp7[i][-1][7:]

In [245]:
temp6

[[1, 1, 0, 1, 1, 1, 0, 1, [0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0]],
 [1, 1, 0, 0, 0, 1, 1, 0, [1]],
 [1, 0, 1, 1, 1, 1, 1, 0, [1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0]],
 [1, 0, 0, 1, 0, 1, 1, 0, [1]],
 [1, 0, 0, 0, 0, 1, 1, 0, [1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0]],
 [1, 0, 0, 1, 0, 1, 1, 1, [1]],
 [1, 0, 0, 0, 1, 1, 0, 1, [1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0]],
 [1, 0, 0, 0, 0, 0, 1, 1, [1]],
 [1, 0, 0, 1, 1, 1, 0, 0, [1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0]],
 [1, 0, 0, 0, 0, 0, 0, 0, [1]],
 [1, 1, 0, 1, 0, 0, 1, 0, [1]],
 [1, 1, 1, 1, 0, 1, 0, 1, [0]],
 [1, 1, 0, 1, 0, 1, 0, 1, [0]],
 [0, 0, 0, 1, 1, 0, 0, 0, [1]],
 [1, 1, 0, 1, 0, 0, 0, 1, [1]],
 [1, 0, 1, 1, 1, 1, 1, 1, [0]],
 [1, 1, 1, 1, 0, 0, 0, 1, [0]],
 [1, 0, 0, 1, 0, 1, 0, 1, [1]],
 [0, 0, 0, 1, 0, 1, 1, 0, [1]],
 [1, 1, 1, 0, 0, 1, 1, 0, [0]],
 [1, 1, 1, 1, 1, 1, 0, 1, [0]],
 [1, 0, 

In [151]:
# Read JSON file
with open('../network/network.json') as data_file:
    network2 = json.load(data_file)

In [32]:
seed(1)
network = initialize_network([8,10,9,8])

In [35]:

# n_inputs = len(temp1[0]) - 1
# n_outputs = len(temp1[0][-1])
start = timeit.default_timer()
train_network(network, dataset, 1.0, 10, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

time = 0.17639622800197685


In [277]:
c = 0
new_dataset = list()
for i in temp7:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        print(type(k))
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

<class 'str'>
0 %


In [264]:
# Write JSON file
with io.open('../network/decryptor/network_bit_8.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [127]:
seed(1)
network = initialize_network([8,9,7,2])

In [130]:
seed(1)
network = initialize_network([8,9,7,2])
n_inputs = len(temp3[0]) - 1
n_outputs = len(temp3[0][-1])
start = timeit.default_timer()
train_network(network, temp3[0:50], 1.0, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=1.000, error=1.001
> epoch=199, lrate=1.000, error=1.001
> epoch=299, lrate=1.000, error=1.000
> epoch=399, lrate=1.000, error=1.000
> epoch=499, lrate=1.000, error=1.000
> epoch=599, lrate=1.000, error=1.000
> epoch=699, lrate=1.000, error=1.000
> epoch=799, lrate=1.000, error=1.000
> epoch=899, lrate=1.000, error=1.000
> epoch=999, lrate=1.000, error=1.000
> epoch=1099, lrate=1.000, error=1.000
> epoch=1199, lrate=1.000, error=1.000
> epoch=1299, lrate=1.000, error=1.000
> epoch=1399, lrate=1.000, error=1.000
> epoch=1499, lrate=1.000, error=1.000
> epoch=1599, lrate=1.000, error=1.000
> epoch=1699, lrate=1.000, error=1.000
> epoch=1799, lrate=1.000, error=1.000
> epoch=1899, lrate=1.000, error=1.000
> epoch=1999, lrate=1.000, error=1.000
> epoch=2099, lrate=1.000, error=1.000
> epoch=2199, lrate=1.000, error=1.000
> epoch=2299, lrate=1.000, error=1.000
> epoch=2399, lrate=1.000, error=1.000
> epoch=2499, lrate=1.000, error=1.000
> epoch=2599, lrate=1.000, error=1.0

In [129]:
c = 0
new_dataset = list()
for i in temp3:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

3 %


In [45]:
# Write JSON file
with io.open('../network/decryptor/network2.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [65]:
# Read JSON file
with open('../network/decryptor/network_bit_2.json') as data_file:
    network2 = json.load(data_file)
    
with open('../network/decryptor/network_bit_3.json') as data_file:
    network3 = json.load(data_file)
    
with open('../network/decryptor/network_bit_4.json') as data_file:
    network4 = json.load(data_file)
    
with open('../network/decryptor/network_bit_5.json') as data_file:
    network5 = json.load(data_file)
    
with open('../network/decryptor/network_bit_6.json') as data_file:
    network6 = json.load(data_file)
    
with open('../network/decryptor/network_bit_7.json') as data_file:
    network7 = json.load(data_file)
    
with open('../network/decryptor/network_bit_8.json') as data_file:
    network8 = json.load(data_file)
   


Reading encrypted file and decrypting it

In [66]:
file = open('../asciiTextData/encrypted.txt','r')
text = file.read()
file.close()

In [67]:
text

'eNqtVIENwzAIe8n8/9yqFYOdhKrS1qnJlhFjMAB8nwi+14PQ/Vrvc1m/xnmee560HT8AYcskX9TdHa/PySr5FG6C3R4aE/nXwqPwNp75Ve6LX7Rf5gf2m/YAUetSLsQPYOB3PC//zi9UpfIdskusDBaSt9bX/ZzzZ7yjg1T+vGxaUrEpX5JfRgVJSNWH6eQ8Mj7DR7mW/B3xomw9DgjWL/iC806fOb539bXUx5rPqa/RffWo80HHZOZdiYnPVP/oCRGH+lvnyda37Rell82fkc/zXJj6cptPXh8S11EfOQd0Hm5z6VU//Ku+OkjIfI22lWikt6H6uf/GQ/dNRawNJoNoqXPBFz40okC+3/c+XlZMGg=='

In [68]:
t = "eNqtVIENwzAIe8n8/9yqFYOdhKrS1qnJlhFjMAB8nwi+14PQ/Vrvc1m/xnmee560HT8AYcskX9TdHa/PySr5FG6C3R4aE/nXwqPwNp75Ve6LX7Rf5gf2m/YAUetSLsQPYOB3PC//zi9UpfIdskusDBaSt9bX/ZzzZ7yjg1T+vGxaUrEpX5JfRgVJSNWH6eQ8Mj7DR7mW/B3xomw9DgjWL/iC806fOb539bXUx5rPqa/RffWo80HHZOZdiYnPVP/oCRGH+lvnyda37Rell82fkc/zXJj6cptPXh8S11EfOQd0Hm5z6VU//Ku+OkjIfI22lWikt6H6uf/GQ/dNRawNJoNoqXPBFz40okC+3/c+XlZMGg=="

In [69]:
text == t

True

In [70]:
decoded_txt = zlib.decompress(base64.b64decode(text))
output = decoded_txt.decode()

In [71]:
output

'000000011001100111110101111101011101000111100111100111001101000111010001101101001110011110101010100000111110011010011001100101010001100011010001110100010001100011100111111101011010101010110100101000011001100100010110111001111010101000011000101101001110011110011100100110010001100011100110111101011001100111010101100110010001100000010011101101111011110111011100101000010001011011100111101000010001011011100111111001101101110010011001111001111101000110000011100000111010000110001101101000011010101011110101111001111110011010011001000101101110011010100001000110001001110011100111101101001101000110001101111111011101010110011001000110001110011011100111100000111101000110010101111001111001100100011000100011011001010110011000101111111110011011010001100101011110011111001011111001111011010010011001100011011001010110011000101111111110011011010001100101011110011110011001100000111000001110100001100011011010000110011001000110001000110110011000000100111011011110111101110111001001100111100111100000111101000

In [72]:
iop = "000000011001100111110101111101011101000111100111100111001101000111010001101101001110011110101010100000111110011010011001100101010001100011010001110100010001100011100111111101011010101010110100101000011001100100010110111001111010101000011000101101001110011110011100100110010001100011100110111101011001100111010101100110010001100000010011101101111011110111011100101000010001011011100111101000010001011011100111111001101101110010011001111001111101000110000011100000111010000110001101101000011010101011110101111001111110011010011001000101101110011010100001000110001001110011100111101101001101000110001101111111011101010110011001000110001110011011100111100000111101000110010101111001111001100100011000100011011001010110011000101111111110011011010001100101011110011111001011111001111011010010011001100011011001010110011000101111111110011011010001100101011110011110011001100000111000001110100001100011011010000110011001000110001000110110011000000100111011011110111101110111001001100111100111100000111101000111110101111101011101000110110101101000010001100010011100111001111101010110011001000101100001011010101010100111001001100111100111101000010001011011100111100000001111110100010110111001101110011110101010111001111001110010101010100101011001011110101010100111001001100111100111101010100001100010110100111001111010000100010110111001111101000100011000111101011001100011100111100011011101110010011001100011011101001010011001101101001110011110000011110100011001010111100111100110011000001110000011101000011000110110100001100110010001100010001101100110001011011100001010101101111000100010110111011111111101110010000000111100111010000110110111000000001011111010100110110001101100001111110010000100111011010010000000101101111000000010010000100000001000000010110100"

In [73]:
output == iop

True

In [74]:
if len(output)%8 != 0:
    print("corrupted data")

In [75]:
c = 0
l = len(output)
k = int(l/8)
s = ""
for i in range(k):
    word = list(output[c:c+8])
    word = list(map(int,output[c:c+8]))
    word.append(0)
    s += "0"
    s += predict(network2,word,2)
    s += predict(network3,word,3)
    s += predict(network4,word,2)
    s += predict(network5,word,2)
    s += predict(network6,word,2)
    s += predict(network7,word,2)
    s += predict(network8,word,2)
    c += 8

In [76]:
s

'010010000110010101101100011011000110111100100000011001110110111101101111011001000010000001100001011001100111010001100101011100100110111001101111011011110110111000100000011011000110000101100100011010010110010101110011001000000110000101101110011001000010000001100111011001010110111001110100011011000110010101101101011001010110111000101110000010100101010001101000011010010111001100100000011010010111001100100000011101000110100001100101001000000110111101100110011001100110100101100011011010010110000101101100001000000111010001100101011100110111010001101001011011100110011100100000011001000110111101100011011101010110110101100101011011100111010000100000011001100110111101110010001000000110010101101110011000110111001001111001011100000111010001101111011100100010000000100110001000000110010001100101011000110111001001111001011100000111010001101111011100100010000001100101011001100110011001101001011000110110100101100101011011100110001101111001001011100000101001010100011010000110010100100000011001100110111

In [63]:
p = "010001000111001001110110011101100111011100110000011100110111011101110111011100100011000001110000011100110111101001110010011110010111011101110111011101110111011100110000011101100111000001110010011101000111001001111001001100000111000001110111011100100011000001110011011100100111011101111010011101100111001001110110011100100111011100110111000001010100101001110100011101000111100100110000011101000111100100110000011110100111010001110010001100000111011101110011011100110111010001110001011101000111000001110110001100000111101001110010011110010111101001110100011101110111001100110000011100100111011101110001011110100111011001110010011101110111101000110000011100110111011101111001001100000111001001110111011100010111100101111100011110000111101001110111011110010011000000110011001100000111001001110010011100010111100101111100011110000111101001110111011110010011000001110010011100110111001101110100011100010111010001110010011101110111000101111100001101110000010101001010011101000111001000110000011100110111011101110110011101100111011101111011011101000111011101110011001100000111011001110010011110010111100101110000011100110111001000110000011101000111100100110000011101010111101001111001011110100011000001110000001100000111001101110000011110010111000101110000011100110111001000110000011100000111011101110010001100000111010001111001001100000111011101110111011101100111110000110000011100010111010001110010011100010111010101110010011100100011000001110011011101110111100100110000011100100111001101110011011101000111000101110100011100100111011101110001011111000000010101000000000001010100001100000101010000010111010001110101010010100111010000000101001100100011100100111010001110010100000000110111001101110111001001110101000001010111010101001001011101010111010101110010"

In [77]:
p

'010001000111001001110110011101100111011100110000011100110111011101110111011100100011000001110000011100110111101001110010011110010111011101110111011101110111011100110000011101100111000001110010011101000111001001111001001100000111000001110111011100100011000001110011011100100111011101111010011101100111001001110110011100100111011100110111000001010100101001110100011101000111100100110000011101000111100100110000011110100111010001110010001100000111011101110011011100110111010001110001011101000111000001110110001100000111101001110010011110010111101001110100011101110111001100110000011100100111011101110001011110100111011001110010011101110111101000110000011100110111011101111001001100000111001001110111011100010111100101111100011110000111101001110111011110010011000000110011001100000111001001110010011100010111100101111100011110000111101001110111011110010011000001110010011100110111001101110100011100010111010001110010011101110111000101111100001101110000010101001010011101000111001000110000011100110111011

In [49]:
"010001000111001001110110011101100111011100110000011100110111011101110111011100100011000001110000011100110111101001110010011110010111011101110111011101110111011100110000011101100111000001110010011101000111001001111001001100000111000001110111011100100011000001110011011100100111011101111010011101100111001001110110011100100111011100110111000001010100101001110100011101000111100100110000011101000111100100110000011110100111010001110010001100000111011101110011011100110111010001110001011101000111000001110110001100000111101001110010011110010111101001110100011101110111001100110000011100100111011101110001011110100111011001110010011101110111101000110000011100110111011101111001001100000111001001110111011100010111100101111100011110000111101001110111011110010011000000110011001100000111001001110010011100010111100101111100011110000111101001110111011110010011000001110010011100110111001101110100011100010111010001110010011101110111000101111100001101110000010101001010011101000111001000110000011100110111011101110110011101100111011101111011011101000111011101110011001100000111011001110010011110010111100101110000011100110111001000110000011101000111100100110000011101010111101001111001011110100011000001110000001100000111001101110000011110010111000101110000011100110111001000110000011100000111011101110010001100000111010001111001001100000111011101110111011101100111110000110000011100010111010001110010011100010111010101110010011100100011000001110011011101110111100100110000011100100111001101110011011101000111000101110100011100100111011101110001011111000000010101000000000001010100001100000101010000010111010001110101010010100111010000000101001100100011100100111010001110010100000000110111001101110111001001110101000001010111010101001001011101010111010101110010"

'010001000111001001110110011101100111011100110000011100110111011101110111011100100011000001110000011100110111101001110010011110010111011101110111011101110111011100110000011101100111000001110010011101000111001001111001001100000111000001110111011100100011000001110011011100100111011101111010011101100111001001110110011100100111011100110111000001010100101001110100011101000111100100110000011101000111100100110000011110100111010001110010001100000111011101110011011100110111010001110001011101000111000001110110001100000111101001110010011110010111101001110100011101110111001100110000011100100111011101110001011110100111011001110010011101110111101000110000011100110111011101111001001100000111001001110111011100010111100101111100011110000111101001110111011110010011000000110011001100000111001001110010011100010111100101111100011110000111101001110111011110010011000001110010011100110111001101110100011100010111010001110010011101110111000101111100001101110000010101001010011101000111001000110000011100110111011

In [45]:
s == p

False

In [41]:
c = 0
l = len(output)
k = int(l/8)
o = ""
for i in range(k):
    u = text_from_bits(s[c:c+8])
    o += u
    c += 8

In [42]:
o

'Hello good afternoon ladies and gentlemen.\nThis is the official testing document for encryptor & decryptor efficiency.\nThe following message is just a garbage and is only checked for efficiency\nA\nG\nBhjUi\n$342@/.dj\njSjjd'

In [37]:
f=open('../asciiTextData/decrypted.txt','w')
f.write(o)
f.close()

In [38]:
print()

SyntaxError: keyword can't be an expression (<ipython-input-38-a106ee619038>, line 1)